In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, BaggingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, recall_score
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option("display.width", 200)

def repeated_k_fold(model, X, y, n_splits=5, n_repeats=10):
    rkf = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=42)

    accuracy_train, accuracy_test = [], []
    f1, recall, roc_auc = [], [], []

    for train_index, test_index in rkf.split(X, y):
        # Aufteilen in Trainings- und Testdaten
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Skalierung: nur an den Trainingsdaten fitten
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        model.fit(X_train_scaled, y_train)

        y_train_pred = model.predict(X_train_scaled)
        y_test_pred = model.predict(X_test_scaled)

        accuracy_train.append(accuracy_score(y_train, y_train_pred))
        accuracy_test.append(accuracy_score(y_test, y_test_pred))
        f1.append(f1_score(y_test, y_test_pred))
        recall.append(recall_score(y_test, y_test_pred))
        roc_auc.append(roc_auc_score(y_test, model.predict_proba(X_test_scaled)[:, 1]))

    return {
        "Train Accuracy": (np.mean(accuracy_train), np.std(accuracy_train)),
        "Test Accuracy": (np.mean(accuracy_test), np.std(accuracy_test)),
        "F1-Score_1": (np.mean(f1), np.std(f1)),
        "Recall_1": (np.mean(recall), np.std(recall)),
        "ROC-AUC_1": (np.mean(roc_auc), np.std(roc_auc)),
    }


# Pfad zur Datei
file_path = r"K:\Team\Böhmer_Michael\TSA\ML\Basistabelle\Basistabelle_ML_neu.xlsx"

try: 
    df = pd.read_excel(file_path)
    
    # Zielvariable (y) und Features (X) extrahieren
    y = df['Verletzungsstatus']
    
    # Dummy-Variable "Geschlecht_weiblich" separieren, falls vorhanden
    if 'Geschlecht_weiblich' in df.columns:
        geschlecht_weiblich = df[['Geschlecht_weiblich']]
        X = df.drop(columns=['Verletzungsstatus', 'Geschlecht_weiblich'])
        # Die Dummy-Variable wieder anhängen
        X = np.hstack((X.values, geschlecht_weiblich.values))
    else:
        X = df.drop(columns=['Verletzungsstatus']).values
    
    # Modelle definieren
    models = {
        "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
        "Decision Tree": DecisionTreeClassifier(random_state=42),
        "Random Forest": RandomForestClassifier(random_state=42),
        "Gradient Boosting": GradientBoostingClassifier(random_state=42),
        "XGBoost": XGBClassifier(eval_metric="logloss", random_state=42),
        "LightGBM": LGBMClassifier(verbose=-1, random_state=42),
        "SVC": SVC(probability=True, random_state=42),
        "k-Nearest Neighbors": KNeighborsClassifier(),
        "MLP Classifier": MLPClassifier(max_iter=1000, random_state=42),
        "Gaussian Naive Bayes": GaussianNB(),
        "Linear Discriminant Analysis": LinearDiscriminantAnalysis(),
        "Quadratic Discriminant Analysis": QuadraticDiscriminantAnalysis(),
        "Bagging Classifier": BaggingClassifier(random_state=42),
        "Extra Trees": ExtraTreesClassifier(random_state=42),
    }
    
    results = []
    for model_name, model in models.items():
        print(f"Modell wird validiert: {model_name}")
        metrics = repeated_k_fold(model, X, y)
        
        formatted_metrics = {
            "Model": model_name,
            "Train Accuracy": f"{metrics['Train Accuracy'][0]:.4f} ± {metrics['Train Accuracy'][1]:.4f}",
            "Test Accuracy": f"{metrics['Test Accuracy'][0]:.4f} ± {metrics['Test Accuracy'][1]:.4f}",
            "F1-Score_1": f"{metrics['F1-Score_1'][0]:.4f} ± {metrics['F1-Score_1'][1]:.4f}",
            "Recall_1": f"{metrics['Recall_1'][0]:.4f} ± {metrics['Recall_1'][1]:.4f}",
            "ROC-AUC_1": f"{metrics['ROC-AUC_1'][0]:.4f} ± {metrics['ROC-AUC_1'][1]:.4f}",
        }
        results.append(formatted_metrics)
    
    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values(by="ROC-AUC_1", ascending=False)
    
    print("\nErgebnisse der Modelle:")
    print(results_df)


except FileNotFoundError:
    print("Die Datei wurde nicht gefunden. Bitte überprüfe den Pfad.")
except Exception as e:
    print(f"Ein Fehler ist aufgetreten: {e}")


Modell wird validiert: Logistic Regression
Modell wird validiert: Decision Tree
Modell wird validiert: Random Forest
Modell wird validiert: Gradient Boosting
Modell wird validiert: XGBoost
Modell wird validiert: LightGBM
Modell wird validiert: SVC
Modell wird validiert: k-Nearest Neighbors
Modell wird validiert: MLP Classifier
Modell wird validiert: Gaussian Naive Bayes
Modell wird validiert: Linear Discriminant Analysis
Modell wird validiert: Quadratic Discriminant Analysis


C:\Users\boehmer\AppData\Local\anaconda3\Lib\site-packages\sklearn\discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\boehmer\AppData\Local\anaconda3\Lib\site-packages\sklearn\discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\boehmer\AppData\Local\anaconda3\Lib\site-packages\sklearn\discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\boehmer\AppData\Local\anaconda3\Lib\site-packages\sklearn\discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\boehmer\AppData\Local\anaconda3\Lib\site-packages\sklearn\discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\boehmer\AppData\Local\anaconda3\Lib\site-packages\sklearn\discriminant_analysis.py:935: UserWarning: Varia

Modell wird validiert: Bagging Classifier
Modell wird validiert: Extra Trees

Ergebnisse der Modelle:
                              Model   Train Accuracy    Test Accuracy       F1-Score_1         Recall_1        ROC-AUC_1
0               Logistic Regression  1.0000 ± 0.0000  0.8895 ± 0.0572  0.8953 ± 0.0589  0.8914 ± 0.1000  0.9541 ± 0.0461
8                    MLP Classifier  1.0000 ± 0.0000  0.8604 ± 0.0590  0.8651 ± 0.0641  0.8458 ± 0.1053  0.9441 ± 0.0471
4                           XGBoost  1.0000 ± 0.0000  0.8416 ± 0.0656  0.8454 ± 0.0701  0.8162 ± 0.1068  0.9328 ± 0.0376
6                               SVC  0.9777 ± 0.0090  0.8381 ± 0.0692  0.8502 ± 0.0653  0.8559 ± 0.0961  0.9325 ± 0.0454
13                      Extra Trees  1.0000 ± 0.0000  0.8153 ± 0.0775  0.8263 ± 0.0768  0.8214 ± 0.1040  0.9268 ± 0.0523
2                     Random Forest  1.0000 ± 0.0000  0.8180 ± 0.0746  0.8291 ± 0.0713  0.8232 ± 0.0997  0.9209 ± 0.0486
3                 Gradient Boosting  1.0000 ± 0.000

Datenaugmentation (Datenleakage gänzlich vermeiden)

In [5]:
import pandas as pd
import numpy as np
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import squareform


def perform_clustering(X_sub, max_clusters=4):
    """
    Führt eine Clusteranalyse auf den numerischen Parametern (außer den
    Gruppierungsvariablen "Verletzungsstatus" und "Geschlecht_weiblich")
    durch und teilt diese in max_clusters Gruppen ein.
    """
    # Wähle alle numerischen Spalten außer "Verletzungsstatus" und "Geschlecht_weiblich"
    cols = [col for col in X_sub.columns if col not in ["Verletzungsstatus", "Geschlecht_weiblich"]]
    X_num = X_sub[cols]
    
    # Berechne die Korrelationsmatrix und transformiere sie in eine Distanzmatrix
    corr = X_num.corr().abs()
    dist = np.clip(1 - corr, a_min=0, a_max=None)  # Clippe negative Werte auf 0
    
    # Konvertiere die Distanzmatrix in ein 1D-Array (upper triangular)
    dists = squareform(dist.values)
    
    # Hierarchische Clusteranalyse (z. B. mit der Ward-Methode)
    Z = linkage(dists, method='ward')
    
    # Führe fcluster aus, um die Spalten in max_clusters Cluster zu unterteilen
    cluster_labels = fcluster(Z, max_clusters, criterion='maxclust')
    
    # Ordne den Spalten Clusterlabels zu
    clusters = {}
    for col, label in zip(cols, cluster_labels):
        clusters.setdefault(label, []).append(col)
    return clusters


def augment_subgroup(X_sub, clusters, p_augment=0.7):
    """
    Augmentiert die Daten in X_sub (einen DataFrame für eine Subgruppe)
    für alle in clusters definierten Parametergruppen.
    
    Für jede Zeile und für jede Cluster-Gruppe wird mit Wahrscheinlichkeit p_augment
    ein neuer Wert gezogen – ansonsten wird der Originalwert beibehalten.
    Dabei werden für jede Parametergruppe der Mittelwertvektor und die Kovarianzmatrix
    des gesamten Subdatensatzes berechnet.
    
    Rückgabe: DataFrame mit augmentierten Zeilen (1 augmentiertes Sample pro Originalzeile)
    """
    augmented_rows = []
    
    # Für jede Cluster-Gruppe: Berechne Mittelwert und Kovarianzmatrix
    cluster_params = {}
    for cl_id, cols in clusters.items():
        cluster_data = X_sub[cols]
        mu = cluster_data.mean().values
        # Falls nur ein Feature im Cluster ist, gibt np.cov einen Skalar zurück.
        if len(cols) == 1:
            cov = np.cov(cluster_data.values.flatten(), ddof=0)
            cov = np.atleast_2d(cov)
        else:
            cov = np.cov(cluster_data.values, rowvar=False)
        cluster_params[cl_id] = (mu, cov, cols)
    
    # Gehe jede Zeile (Originaldatensatz) durch
    for idx, row in X_sub.iterrows():
        new_row = row.copy()
        # Für jede Cluster-Gruppe stochastisch augmentieren
        for cl_id, (mu, cov, cols) in cluster_params.items():
            if np.random.rand() < p_augment:
                # Ziehe einen neuen Vektor aus der multivariaten Normalverteilung
                new_values = np.random.multivariate_normal(mu, cov)
                # Überschreibe nur die Werte in den entsprechenden Spalten
                for col, val in zip(cols, new_values):
                    new_row[col] = val
        augmented_rows.append(new_row)
    
    augmented_df = pd.DataFrame(augmented_rows)
    
    # Sicherstellen, dass Gruppierungsvariablen den richtigen Datentyp behalten:
    for col in ["Geschlecht_weiblich", "Verletzungsstatus"]:
        if col in augmented_df.columns:
            augmented_df[col] = augmented_df[col].astype(int)
    
    return augmented_df


# ----------------------------------------------------------------
# Funktion: Stratifikation und Augmentation der Trainingsdaten
# ----------------------------------------------------------------
def augment_training_data(X_train, y_train, max_clusters=4, p_augment=0.7, num_new_samples=1):
    """
    Führt zuerst eine Stratifikation des Trainingsdatensatzes nach "Verletzungsstatus"
    und "Geschlecht_weiblich" durch und wendet dann in jeder Subgruppe:
      - eine Clusteranalyse (auf alle übrigen numerischen Features)
      - die Augmentation (Ziehung neuer Samples aus multivariater Normalverteilung)
    an.
    
    num_new_samples gibt an, wie viele augmentierte Samples pro Originalzeile generiert werden.
    """
    # Kombiniere X_train und y_train, damit wir gruppieren können
    df_train = X_train.copy()
    df_train["Verletzungsstatus"] = y_train
    df_train.reset_index(drop=True, inplace=True)
    
    # Stratifikation: Erstelle eine Gruppierung nach (Verletzungsstatus, Geschlecht_weiblich)
    augmented_groups = []
    for key, group_df in df_train.groupby(["Verletzungsstatus", "Geschlecht_weiblich"]):
        group_df = group_df.reset_index(drop=True)
        # Führe Clusteranalyse auf dieser Subgruppe durch
        clusters = perform_clustering(group_df, max_clusters=max_clusters)
        # Erzeuge für diese Subgruppe num_new_samples augmentierte Samples pro Zeile
        aug_list = []
        for _ in range(num_new_samples):
            aug = augment_subgroup(group_df, clusters, p_augment=p_augment)
            aug_list.append(aug)
        aug_group = pd.concat(aug_list, axis=0)
        augmented_groups.append(aug_group)
    
    # Vereinige alle augmentierten Subgruppen
    augmented_df = pd.concat(augmented_groups, axis=0).reset_index(drop=True)
    
    # Trenne Zielvariable und Features (hier war "Verletzungsstatus" in df_train enthalten)
    y_aug = augmented_df["Verletzungsstatus"]
    X_aug = augmented_df.drop(columns=["Verletzungsstatus"])
    
    return X_aug, y_aug

# ----------------------------------------------------------------
# Integration in den Trainingssplit (innerer CV) mit Augmentation
# ----------------------------------------------------------------
def inner_cv_with_augmentation(X_train, y_train):
    from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
    from sklearn.linear_model import LogisticRegression

    inner_cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)
    scores = []

    for train_idx, test_idx in inner_cv.split(X_train, y_train):
        # Kopien der Daten erstellen
        X_inner_train = X_train.iloc[train_idx].copy()
        X_inner_test = X_train.iloc[test_idx].copy()
        y_inner_train, y_inner_test = y_train.iloc[train_idx], y_train.iloc[test_idx]

        # Normalisierung innerhalb des Folds:
        dummy = "Geschlecht_weiblich"
        cols_to_scale = [col for col in X_inner_train.columns if col != dummy]
        scaler = StandardScaler()
        X_inner_train[cols_to_scale] = scaler.fit_transform(X_inner_train[cols_to_scale])
        X_inner_test[cols_to_scale] = scaler.transform(X_inner_test[cols_to_scale])

        # Augmentation auf den normalisierten Trainingsdaten durchführen:
        X_inner_train_aug, y_inner_train_aug = augment_training_data(
            X_inner_train, y_inner_train, max_clusters=4, p_augment=0.7, num_new_samples=3
        )

        model = LogisticRegression(max_iter=1000)
        # Auch hier wird cross_val_score innerhalb der Trainingsdaten-CV eingesetzt
        score = cross_val_score(model, X_inner_train_aug, y_inner_train_aug, cv=3,
                                scoring="roc_auc", n_jobs=-1).mean()
        scores.append(score)

    avg_score = np.mean(scores)
    #print("Durchschnittliche innere CV-ROC-AUC mit Augmentation:", avg_score)
    return avg_score

#------------------------------------------------------------------------
# Integration in den Trainingssplit (innerer CV) ohne Augmentation
#------------------------------------------------------------------------
def inner_cv_without_augmentation(X_train, y_train):
    from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
    from sklearn.linear_model import LogisticRegression

    inner_cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)
    scores = []

    for train_idx, test_idx in inner_cv.split(X_train, y_train):
        # Kopien der Daten erstellen, um spätere Änderungen lokal zu halten
        X_inner_train = X_train.iloc[train_idx].copy()
        X_inner_test = X_train.iloc[test_idx].copy()
        y_inner_train, y_inner_test = y_train.iloc[train_idx], y_train.iloc[test_idx]

        # Normalisierung innerhalb des Folds:
        dummy = "Geschlecht_weiblich"
        cols_to_scale = [col for col in X_inner_train.columns if col != dummy]
        scaler = StandardScaler()
        X_inner_train[cols_to_scale] = scaler.fit_transform(X_inner_train[cols_to_scale])
        X_inner_test[cols_to_scale] = scaler.transform(X_inner_test[cols_to_scale])

        model = LogisticRegression(max_iter=1000)
        # cross_val_score führt hier eine weitere CV auf den inneren Trainingsdaten durch
        score = cross_val_score(model, X_inner_train, y_inner_train, cv=3,
                                scoring="roc_auc", n_jobs=-1).mean()
        scores.append(score)
    
    avg_score = np.mean(scores)
    #print("Durchschnittliche innere CV-ROC-AUC ohne Augmentation:", avg_score)
    return avg_score


In [9]:
# Originaldaten laden
file_path = r"K:\Team\Böhmer_Michael\TSA\ML\Basistabelle\Basistabelle_ML_neu.xlsx"
df_original = pd.read_excel(file_path)

# Aufteilen in Features und Zielvariable
X_original = df_original.drop(columns=["Verletzungsstatus"])
y_original = df_original["Verletzungsstatus"]

# Vergleich beider Ansätze:
print("=== Vergleich: Innere CV ohne vs. mit Augmentation ===")
score_without = inner_cv_without_augmentation(X_original, y_original)
score_with = inner_cv_with_augmentation(X_original, y_original)

print("\nZusammenfassung der Ergebnisse:")
print("Durchschnittliche innere CV-ROC-AUC ohne Augmentation:", score_without)
print("Durchschnittliche innere CV-ROC-AUC mit Augmentation:", score_with)
print("Differenz:", score_with - score_without)


=== Vergleich: Innere CV ohne vs. mit Augmentation ===
Durchschnittliche innere CV-ROC-AUC ohne Augmentation: 0.9261234109028226
Durchschnittliche innere CV-ROC-AUC mit Augmentation: 0.9763723972941868

Zusammenfassung der Ergebnisse:
ROC-AUC ohne Augmentation: 0.9261234109028226
ROC-AUC mit Augmentation: 0.9763723972941868
Differenz: 0.050248986391364214


Test-Datenaugmentation-Cluster

Test Cluster-Funktion

In [ ]:
def perform_clustering(X_sub, max_clusters=4):
    """
    Führt eine Clusteranalyse auf den numerischen Parametern (außer den
    Gruppierungsvariablen "Verletzungsstatus" und "Geschlecht_weiblich")
    durch und teilt diese in max_clusters Gruppen ein.
    """
    # Wähle alle numerischen Spalten außer "Verletzungsstatus" und "Geschlecht_weiblich"
    cols = [col for col in X_sub.columns if col not in ["Verletzungsstatus", "Geschlecht_weiblich"]]
    X_num = X_sub[cols]
    
    # Berechne die Korrelationsmatrix und transformiere sie in eine Distanzmatrix
    corr = X_num.corr().abs()
    dist = np.clip(1 - corr, a_min=0, a_max=None)  # Clippe negative Werte auf 0
    
    # Konvertiere die Distanzmatrix in ein 1D-Array (upper triangular)
    dists = squareform(dist.values)
    
    # Hierarchische Clusteranalyse (z. B. mit der Ward-Methode)
    Z = linkage(dists, method='ward')
    
    # Führe fcluster aus, um die Spalten in max_clusters Cluster zu unterteilen
    cluster_labels = fcluster(Z, max_clusters, criterion='maxclust')
    
    # Ordne den Spalten Clusterlabels zu
    clusters = {}
    for col, label in zip(cols, cluster_labels):
        clusters.setdefault(label, []).append(col)
    return clusters

Test Subgruppen 

In [ ]:
def augment_subgroup(X_sub, clusters, p_augment=0.7):
    """
    Augmentiert die Daten in X_sub (einen DataFrame für eine Subgruppe)
    für alle in clusters definierten Parametergruppen.
    
    Für jede Zeile und für jede Cluster-Gruppe wird mit Wahrscheinlichkeit p_augment
    ein neuer Wert gezogen – ansonsten wird der Originalwert beibehalten.
    Dabei werden für jede Parametergruppe der Mittelwertvektor und die Kovarianzmatrix
    des gesamten Subdatensatzes berechnet.
    
    Rückgabe: DataFrame mit augmentierten Zeilen (1 augmentiertes Sample pro Originalzeile)
    """
    augmented_rows = []
    
    # Für jede Cluster-Gruppe: Berechne Mittelwert und Kovarianzmatrix
    cluster_params = {}
    for cl_id, cols in clusters.items():
        cluster_data = X_sub[cols]
        mu = cluster_data.mean().values
        # Falls nur ein Feature im Cluster ist, gibt np.cov einen Skalar zurück.
        if len(cols) == 1:
            cov = np.cov(cluster_data.values.flatten(), ddof=0)
            cov = np.atleast_2d(cov)
        else:
            cov = np.cov(cluster_data.values, rowvar=False)
        cluster_params[cl_id] = (mu, cov, cols)
    
    # Gehe jede Zeile (Originaldatensatz) durch
    for idx, row in X_sub.iterrows():
        new_row = row.copy()
        # Für jede Cluster-Gruppe stochastisch augmentieren
        for cl_id, (mu, cov, cols) in cluster_params.items():
            if np.random.rand() < p_augment:
                # Ziehe einen neuen Vektor aus der multivariaten Normalverteilung
                new_values = np.random.multivariate_normal(mu, cov)
                # Überschreibe nur die Werte in den entsprechenden Spalten
                for col, val in zip(cols, new_values):
                    new_row[col] = val
        augmented_rows.append(new_row)
    
    augmented_df = pd.DataFrame(augmented_rows)
    
    # Sicherstellen, dass Gruppierungsvariablen den richtigen Datentyp behalten:
    for col in ["Geschlecht_weiblich", "Verletzungsstatus"]:
        if col in augmented_df.columns:
            augmented_df[col] = augmented_df[col].astype(int)
    
    return augmented_df



Test Daten-Augmentation

In [ ]:
def augment_training_data(X_train, y_train, max_clusters=4, p_augment=0.7, num_new_samples=1):
    """
    Führt zuerst eine Stratifikation des Trainingsdatensatzes nach "Verletzungsstatus"
    und "Geschlecht_weiblich" durch und wendet dann in jeder Subgruppe:
      - eine Clusteranalyse (auf alle übrigen numerischen Features)
      - die Augmentation (Ziehung neuer Samples aus multivariater Normalverteilung)
    an.
    
    num_new_samples gibt an, wie viele augmentierte Samples pro Originalzeile generiert werden.
    """
    # Kombiniere X_train und y_train, damit wir gruppieren können
    df_train = X_train.copy()
    df_train["Verletzungsstatus"] = y_train
    df_train.reset_index(drop=True, inplace=True)
    
    # Stratifikation: Erstelle eine Gruppierung nach (Verletzungsstatus, Geschlecht_weiblich)
    augmented_groups = []
    for key, group_df in df_train.groupby(["Verletzungsstatus", "Geschlecht_weiblich"]):
        group_df = group_df.reset_index(drop=True)
        # Führe Clusteranalyse auf dieser Subgruppe durch
        clusters = perform_clustering(group_df, max_clusters=max_clusters)
        # Erzeuge für diese Subgruppe num_new_samples augmentierte Samples pro Zeile
        aug_list = []
        for _ in range(num_new_samples):
            aug = augment_subgroup(group_df, clusters, p_augment=p_augment)
            aug_list.append(aug)
        aug_group = pd.concat(aug_list, axis=0)
        augmented_groups.append(aug_group)
    
    # Vereinige alle augmentierten Subgruppen
    augmented_df = pd.concat(augmented_groups, axis=0).reset_index(drop=True)
    
    # Trenne Zielvariable und Features (hier war "Verletzungsstatus" in df_train enthalten)
    y_aug = augmented_df["Verletzungsstatus"]
    X_aug = augmented_df.drop(columns=["Verletzungsstatus"])
    
    return X_aug, y_aug


Clusterbildung-Validierung mit Rauschen global und gruppenspezifisch, über 100 wiederholungen

In [ ]:
import pandas as pd
import numpy as np
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import squareform
from sklearn.decomposition import PCA
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
import matplotlib.pyplot as plt
from collections import Counter

# -----------------------------
# 0. Hilfsfunktion: Gruppenschlüssel übersetzen
# -----------------------------
def map_group_label(key):
    """
    Übersetzt den Gruppenschlüssel (Verletzungsstatus, Geschlecht_weiblich) 
    in eine lesbare Bezeichnung.
    
    Verletzungsstatus: 0 -> "unverletzt", 1 -> "verletzt"
    Geschlecht_weiblich: 0 -> "männlich", 1 -> "weiblich"
    """
    status = "verletzt" if key[0] == 1 else "unverletzt"
    gender = "weiblich" if key[1] == 1 else "männlich"
    return f"Gruppe ({status}-{gender})"

# -----------------------------
# 1. Funktion: Rauschen hinzufügen
# -----------------------------
def add_noise_to_data(df, noise_factor=0.1):
    """
    Fügt allen numerischen Spalten (außer "Geschlecht_weiblich" und "Verletzungsstatus")
    Rauschen hinzu, wobei das Rauschen ein Anteil (noise_factor) der ursprünglichen
    Standardabweichung beträgt.
    """
    df_noisy = df.copy()
    numeric_cols = [col for col in df_noisy.select_dtypes(include=['float64', 'int64']).columns 
                    if col not in ["Geschlecht_weiblich", "Verletzungsstatus"]]
    for col in numeric_cols:
        std = df_noisy[col].std()
        noise = np.random.normal(0, std * noise_factor, size=df_noisy.shape[0])
        df_noisy[col] += noise
    return df_noisy

# -----------------------------
# 2. Funktion: Clusterbildung
# -----------------------------
def perform_clustering(X_sub, max_clusters=4):
    """
    Führt eine hierarchische Clusteranalyse auf den numerischen Features (ohne 
    "Geschlecht_weiblich" und "Verletzungsstatus") durch und teilt in max_clusters ein.
    """
    cols = [col for col in X_sub.columns if col not in ["Verletzungsstatus", "Geschlecht_weiblich"]]
    X_num = X_sub[cols]
    corr = X_num.corr().abs()
    dist = np.clip(1 - corr, a_min=0, a_max=None)
    dists = squareform(dist.values)
    Z = linkage(dists, method='ward')
    cluster_labels = fcluster(Z, max_clusters, criterion='maxclust')
    clusters = {}
    for col, label in zip(cols, cluster_labels):
        clusters.setdefault(label, []).append(col)
    return clusters

# -----------------------------
# Hilfsfunktionen für Stabilitätsmetriken und Visualisierung
# -----------------------------
def get_feature_order(df):
    return [col for col in df.columns if col not in ["Verletzungsstatus", "Geschlecht_weiblich"]]

def get_cluster_labels(clusters, feature_order):
    feature_cluster = {}
    for cl, feats in clusters.items():
        for feat in feats:
            feature_cluster[feat] = cl
    return [feature_cluster.get(feat, -1) for feat in feature_order]

def visualize_clusters(df_group, clusters, title="Cluster Visualization"):
    """
    Visualisiert die Cluster der Features mittels PCA (auf der Korrelationsmatrix).
    Es werden keine Textannotationen angezeigt, damit die Grafik übersichtlicher bleibt.
    """
    feature_order = get_feature_order(df_group)
    X_num = df_group[feature_order]
    corr = X_num.corr().values
    pca = PCA(n_components=2)
    coords = pca.fit_transform(corr)
    labels = get_cluster_labels(clusters, feature_order)
    
    plt.figure(figsize=(8,6))
    scatter = plt.scatter(coords[:,0], coords[:,1], c=labels, cmap='viridis', s=100)
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.title(title)
    plt.colorbar(scatter, label="Cluster")
    plt.show()

# -----------------------------
# 3. Funktionen für Gruppierung und Vergleich
# -----------------------------
def get_group_clusters(df):
    """
    Gruppiert den Datensatz anhand von ("Verletzungsstatus", "Geschlecht_weiblich")
    und berechnet für jede Gruppe die Cluster.
    """
    groups = {}
    for key, group in df.groupby(["Verletzungsstatus", "Geschlecht_weiblich"]):
        group = group.reset_index(drop=True)
        clusters = perform_clustering(group, max_clusters=4)
        groups[key] = clusters
    return groups

def compute_stability_metrics(clusters_1, clusters_2, feature_order):
    """
    Vergleicht zwei Clusterlösungen anhand von ARI und NMI.
    """
    labels_1 = get_cluster_labels(clusters_1, feature_order)
    labels_2 = get_cluster_labels(clusters_2, feature_order)
    ari = adjusted_rand_score(labels_1, labels_2)
    nmi = normalized_mutual_info_score(labels_1, labels_2)
    return ari, nmi

def compute_pairwise_metrics(solution_list, feature_order):
    """
    Berechnet pairwise ARI und NMI über eine Liste von Clusterlösungen.
    Gibt (mean_ARI, std_ARI, mean_NMI, std_NMI) zurück.
    """
    aris = []
    nmis = []
    n = len(solution_list)
    for i in range(n):
        for j in range(i+1, n):
            ari, nmi = compute_stability_metrics(solution_list[i], solution_list[j], feature_order)
            aris.append(ari)
            nmis.append(nmi)
    return np.mean(aris), np.std(aris), np.mean(nmis), np.std(nmis)

def compute_pairwise_consistency(solution_list):
    """
    Berechnet pairwise Cluster-Konsistenz (durchschnittlicher Match-Prozentsatz) über eine Liste von Clusterlösungen.
    """
    consistencies = []
    n = len(solution_list)
    for i in range(n):
        for j in range(i+1, n):
            cons = average_cluster_consistency(solution_list[i], solution_list[j])
            consistencies.append(cons)
    return np.mean(consistencies), np.std(consistencies)

def average_cluster_consistency(clusters_1, clusters_2):
    """
    Berechnet für einen Vergleich zweier Clusterlösungen den durchschnittlichen Match-Prozentsatz.
    Für jeden Cluster in clusters_1 wird der beste Matching-Cluster in clusters_2 gesucht.
    """
    match_percentages = []
    for cluster_id, features_1 in clusters_1.items():
        best_match = max(clusters_2.items(), key=lambda x: len(set(features_1).intersection(set(x[1]))))
        match_percentage = len(set(features_1).intersection(set(best_match[1]))) / len(features_1) * 100
        match_percentages.append(match_percentage)
    return np.mean(match_percentages) if match_percentages else np.nan

def get_detailed_cluster_consistency(clusters_1, clusters_2):
    """
    Für jeden Cluster in clusters_1 wird der beste Matching-Cluster in clusters_2 gesucht.
    Gibt ein Dictionary zurück: {orig_cluster: (matched_cluster, match_percentage)}
    """
    details = {}
    for cluster_id, features_1 in clusters_1.items():
        best_match = max(clusters_2.items(), key=lambda x: len(set(features_1).intersection(set(x[1]))))
        match_percentage = len(set(features_1).intersection(set(best_match[1]))) / len(features_1) * 100
        details[cluster_id] = (best_match[0], match_percentage)
    return details

# -----------------------------
# 4. Funktion: Gruppenspezifisches Rauschen hinzufügen
# -----------------------------
def add_noise_to_group(df_group, noise_factor=0.1):
    return add_noise_to_data(df_group, noise_factor=noise_factor)

# -----------------------------
# Simulation: Für eine gegebene Untergruppe
# -----------------------------
def simulate_state_for_group(df_original, key, noise_factor=0.1, n_reps=100, state="global"):
    """
    Simuliert n_reps Clusterlösungen für eine gegebene Subgruppe (key) im Zustand:
      - "global": Es wird global Rauschen zum gesamten Datensatz hinzugefügt.
      - "group": Es wird nur in der Subgruppe Rauschen hinzugefügt.
    Gibt eine Liste von Clusterlösungen zurück.
    """
    solutions = []
    group_orig = df_original[(df_original["Verletzungsstatus"] == key[0]) & (df_original["Geschlecht_weiblich"] == key[1])]
    for _ in range(n_reps):
        if state == "global":
            df_global_noisy = add_noise_to_data(df_original, noise_factor)
            group = df_global_noisy[(df_global_noisy["Verletzungsstatus"] == key[0]) & (df_global_noisy["Geschlecht_weiblich"] == key[1])]
        elif state == "group":
            group = add_noise_to_group(group_orig, noise_factor)
        else:
            raise ValueError("state muss 'global' oder 'group' sein.")
        clusters = perform_clustering(group, max_clusters=4)
        solutions.append(clusters)
    return solutions

# -----------------------------
# 5. Main-Funktion
# -----------------------------
def main():
    # 5.1 Daten laden
    file_path = r"K:\Team\Böhmer_Michael\TSA\ML\Basistabelle\Basistabelle_ML_neu.xlsx"
    df_original = pd.read_excel(file_path)
    print("Original Data Shape:", df_original.shape, "\n")
    
    # Baseline: Original Cluster (deterministisch)
    clusters_original = get_group_clusters(df_original)
    
    # --- ERSTER DURCHLAUF: Ausgabe von Variablenlisten und Visualisierungen ---
    print("==== ERSTER DURCHLAUF: Variablenlisten und Visualisierungen ====\n")
    
    # Global verrauschte Cluster (ein Durchlauf)
    df_global_noisy = add_noise_to_data(df_original, noise_factor=0.1)
    clusters_global_noisy = get_group_clusters(df_global_noisy)
    
    # Gruppenspezifisch verrauschte Cluster (ein Durchlauf)
    clusters_group_specific = {}
    for key, group in df_original.groupby(["Verletzungsstatus", "Geschlecht_weiblich"]):
        group = group.reset_index(drop=True)
        group_noisy = add_noise_to_group(group, noise_factor=0.1)
        clusters_group_specific[key] = perform_clustering(group_noisy, max_clusters=4)
    
    # Ausgabe der Clusterzuordnungen (Variablenlisten)
    print("--- Cluster Zuordnungen: Original ---\n")
    for key, clusters in clusters_original.items():
        print(map_group_label(key))
        for cl, cols in clusters.items():
            print("  Cluster", cl, ":", cols)
        print("")
    
    print("--- Cluster Zuordnungen: Global Noisy ---\n")
    for key, clusters in clusters_global_noisy.items():
        print(map_group_label(key))
        for cl, cols in clusters.items():
            print("  Cluster", cl, ":", cols)
        print("")
    
    print("--- Cluster Zuordnungen: Gruppenspezifisch Noisy ---\n")
    for key, clusters in clusters_group_specific.items():
        print(map_group_label(key))
        for cl, cols in clusters.items():
            print("  Cluster", cl, ":", cols)
        print("")
    
    # Visualisierung für eine Beispieluntergruppe
    example_key = list(clusters_original.keys())[0]
    group_orig = df_original[(df_original["Verletzungsstatus"]==example_key[0]) & 
                             (df_original["Geschlecht_weiblich"]==example_key[1])]
    group_global_noisy = df_global_noisy[(df_global_noisy["Verletzungsstatus"]==example_key[0]) & 
                                         (df_global_noisy["Geschlecht_weiblich"]==example_key[1])]
    group_group_noisy = add_noise_to_group(group_orig, noise_factor=0.1)
    clusters_example = perform_clustering(group_group_noisy, max_clusters=4)
    
    visualize_clusters(group_orig, clusters_original[example_key],
                         title=f"Original Cluster ({map_group_label(example_key)})")
    visualize_clusters(group_global_noisy, clusters_global_noisy[example_key],
                         title=f"Global Noisy Cluster ({map_group_label(example_key)})")
    visualize_clusters(group_group_noisy, clusters_example,
                         title=f"Gruppenspezifisch Noisy Cluster ({map_group_label(example_key)})")
    
    # --- SIMULATION (50 Wiederholungen) zur Aggregation der Metriken ---
    n_reps = 50
    sim_results = {}   # Speichert für jede Subgruppe: { 'original': (ARI, NMI, Consistency),
                      #  'global': (mean_ARI, std_ARI, mean_NMI, std_NMI, mean_cons, std_cons),
                      #  'group': (mean_ARI, std_ARI, mean_NMI, std_NMI, mean_cons, std_cons) }
    sim_details = {}   # Für die detaillierte Konsistenz: {key: {comp: {orig_cluster: {'matched': [], 'consistency': []}} } }
    
    for key in clusters_original.keys():
        sim_results[key] = {
            'original': (1.0, 0.0, 1.0, 0.0, 100.0, 0.0),
            'global': None,
            'group': None
        }
        sim_details[key] = {
            'orig_vs_global': {},
            'orig_vs_group': {},
            'global_vs_group': {}
        }
        
        feature_order = get_feature_order(df_original[(df_original["Verletzungsstatus"]==key[0]) & 
                                                      (df_original["Geschlecht_weiblich"]==key[1])])
        
        # Global verrauscht
        global_solutions = simulate_state_for_group(df_original, key, 0.1, n_reps, state="global")
        ari_g, std_ari_g, nmi_g, std_nmi_g = compute_pairwise_metrics(global_solutions, feature_order)
        cons_g, std_cons_g = compute_pairwise_consistency(global_solutions)
        
        # Gruppenspezifisch verrauscht
        group_solutions = simulate_state_for_group(df_original, key, 0.1, n_reps, state="group")
        ari_grp, std_ari_grp, nmi_grp, std_nmi_grp = compute_pairwise_metrics(group_solutions, feature_order)
        cons_grp, std_cons_grp = compute_pairwise_consistency(group_solutions)
        
        sim_results[key]['global'] = (ari_g, std_ari_g, nmi_g, std_nmi_g, cons_g, std_cons_g)
        sim_results[key]['group'] = (ari_grp, std_ari_grp, nmi_grp, std_nmi_grp, cons_grp, std_cons_grp)
        
        # Aggregiere detaillierte Konsistenzdaten
        for comp, sols in zip(['orig_vs_global', 'orig_vs_group'], [global_solutions, group_solutions]):
            details = [get_detailed_cluster_consistency(clusters_original[key], sol) for sol in sols]
            # Nun berechnen wir für jeden Originalcluster die Liste der matched Werte und Consistencies
            for d in details:
                for orig_cluster, (matched, cons) in d.items():
                    if orig_cluster not in sim_details[key][comp]:
                        sim_details[key][comp][orig_cluster] = {'matched': [], 'consistency': []}
                    sim_details[key][comp][orig_cluster]['matched'].append(matched)
                    sim_details[key][comp][orig_cluster]['consistency'].append(cons)
        
        # Für global_vs_group Vergleich
        details = [get_detailed_cluster_consistency(sol1, sol2) for sol1, sol2 in zip(global_solutions, group_solutions)]
        for d in details:
            for orig_cluster, (matched, cons) in d.items():
                if orig_cluster not in sim_details[key]['global_vs_group']:
                    sim_details[key]['global_vs_group'][orig_cluster] = {'matched': [], 'consistency': []}
                sim_details[key]['global_vs_group'][orig_cluster]['matched'].append(matched)
                sim_details[key]['global_vs_group'][orig_cluster]['consistency'].append(cons)
    
    # Aggregierte Ausgabe der stabilitätsmetriken
    print("=== Aggregierte Stabilitätsmetriken über", n_reps, "Wiederholungen ===\n")
    for key, metrics in sim_results.items():
        print(map_group_label(key))
        orig = metrics['original']
        print(f"  original:             ARI = {orig[0]:.3f} ± {orig[1]:.3f}, NMI = {orig[2]:.3f} ± {orig[3]:.3f}, Consistency = {orig[4]:.2f}% ± {orig[5]:.2f}%")
        glob = metrics['global']
        print(f"  global verrauscht:    ARI = {glob[0]:.3f} ± {glob[1]:.3f}, NMI = {glob[2]:.3f} ± {glob[3]:.3f}, Consistency = {glob[4]:.2f}% ± {glob[5]:.2f}%")
        grp = metrics['group']
        print(f"  gruppenspezifisch:    ARI = {grp[0]:.3f} ± {grp[1]:.3f}, NMI = {grp[2]:.3f} ± {grp[3]:.3f}, Consistency = {grp[4]:.2f}% ± {grp[5]:.2f}%")
        print("")
    
    # Aggregierte Ausgabe der detaillierten Cluster-Konsistenz
    print("=== Aggregierte Detaillierte Cluster-Konsistenz (Übereinstimmungsprozente) ===\n")
    for key, comp_dict in sim_details.items():
        print(map_group_label(key))
        for comp, clusters_dict in comp_dict.items():
            print(f"  {comp}:")
            for orig_cluster, values in clusters_dict.items():
                cons_mean = np.mean(values['consistency'])
                cons_std = np.std(values['consistency'])
                mode = Counter(values['matched']).most_common(1)[0][0]
                print(f"    Orig. Cluster {orig_cluster}: {mode} (Übereinstimmung: {cons_mean:.2f}% ± {cons_std:.2f}%)")
            print("")
        print("")
    
if __name__ == '__main__':
    main()


Clusterbildung-Validierung mit Rauschen global und gruppenspezifisch, über 100 wiederholungen

In [ ]:
import pandas as pd
import numpy as np
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import squareform
from sklearn.decomposition import PCA
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
import matplotlib.pyplot as plt
from collections import Counter

# -----------------------------
# 0. Hilfsfunktion: Gruppenschlüssel übersetzen
# -----------------------------
def map_group_label(key):
    """
    Übersetzt den Gruppenschlüssel (Verletzungsstatus, Geschlecht_weiblich) 
    in eine lesbare Bezeichnung.
    
    Verletzungsstatus: 0 -> "unverletzt", 1 -> "verletzt"
    Geschlecht_weiblich: 0 -> "männlich", 1 -> "weiblich"
    """
    status = "verletzt" if key[0] == 1 else "unverletzt"
    gender = "weiblich" if key[1] == 1 else "männlich"
    return f"Gruppe ({status}-{gender})"

# -----------------------------
# 1. Funktion: Rauschen hinzufügen
# -----------------------------
def add_noise_to_data(df, noise_factor=0.1):
    """
    Fügt allen numerischen Spalten (außer "Geschlecht_weiblich" und "Verletzungsstatus")
    Rauschen hinzu, wobei das Rauschen ein Anteil (noise_factor) der ursprünglichen
    Standardabweichung beträgt.
    """
    df_noisy = df.copy()
    numeric_cols = [col for col in df_noisy.select_dtypes(include=['float64', 'int64']).columns 
                    if col not in ["Geschlecht_weiblich", "Verletzungsstatus"]]
    for col in numeric_cols:
        std = df_noisy[col].std()
        noise = np.random.normal(0, std * noise_factor, size=df_noisy.shape[0])
        df_noisy[col] += noise
    return df_noisy

# -----------------------------
# 2. Funktion: Clusterbildung
# -----------------------------
def perform_clustering(X_sub, max_clusters=4):
    """
    Führt eine hierarchische Clusteranalyse auf den numerischen Features (ohne 
    "Geschlecht_weiblich" und "Verletzungsstatus") durch und teilt in max_clusters ein.
    """
    cols = [col for col in X_sub.columns if col not in ["Verletzungsstatus", "Geschlecht_weiblich"]]
    X_num = X_sub[cols]
    corr = X_num.corr().abs()
    dist = np.clip(1 - corr, a_min=0, a_max=None)
    dists = squareform(dist.values)
    Z = linkage(dists, method='ward')
    cluster_labels = fcluster(Z, max_clusters, criterion='maxclust')
    clusters = {}
    for col, label in zip(cols, cluster_labels):
        clusters.setdefault(label, []).append(col)
    return clusters

# -----------------------------
# Hilfsfunktionen für Stabilitätsmetriken und Visualisierung
# -----------------------------
def get_feature_order(df):
    return [col for col in df.columns if col not in ["Verletzungsstatus", "Geschlecht_weiblich"]]

def get_cluster_labels(clusters, feature_order):
    feature_cluster = {}
    for cl, feats in clusters.items():
        for feat in feats:
            feature_cluster[feat] = cl
    return [feature_cluster.get(feat, -1) for feat in feature_order]

def visualize_clusters(df_group, clusters, title="Cluster Visualization"):
    """
    Visualisiert die Cluster der Features mittels PCA (auf der Korrelationsmatrix).
    Es werden keine Textannotationen angezeigt, damit die Grafik übersichtlicher bleibt.
    """
    feature_order = get_feature_order(df_group)
    X_num = df_group[feature_order]
    corr = X_num.corr().values
    pca = PCA(n_components=2)
    coords = pca.fit_transform(corr)
    labels = get_cluster_labels(clusters, feature_order)
    
    plt.figure(figsize=(8,6))
    scatter = plt.scatter(coords[:,0], coords[:,1], c=labels, cmap='viridis', s=100)
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.title(title)
    plt.colorbar(scatter, label="Cluster")
    plt.show()

# -----------------------------
# 3. Funktionen für Gruppierung und Vergleich
# -----------------------------
def get_group_clusters(df):
    """
    Gruppiert den Datensatz anhand von ("Verletzungsstatus", "Geschlecht_weiblich")
    und berechnet für jede Gruppe die Cluster.
    """
    groups = {}
    for key, group in df.groupby(["Verletzungsstatus", "Geschlecht_weiblich"]):
        group = group.reset_index(drop=True)
        clusters = perform_clustering(group, max_clusters=4)
        groups[key] = clusters
    return groups

def compute_stability_metrics(clusters_1, clusters_2, feature_order):
    """
    Vergleicht zwei Clusterlösungen anhand von ARI und NMI.
    """
    labels_1 = get_cluster_labels(clusters_1, feature_order)
    labels_2 = get_cluster_labels(clusters_2, feature_order)
    ari = adjusted_rand_score(labels_1, labels_2)
    nmi = normalized_mutual_info_score(labels_1, labels_2)
    return ari, nmi

def compute_pairwise_metrics(solution_list, feature_order):
    """
    Berechnet pairwise ARI und NMI über eine Liste von Clusterlösungen.
    Gibt (mean_ARI, std_ARI, mean_NMI, std_NMI) zurück.
    """
    aris = []
    nmis = []
    n = len(solution_list)
    for i in range(n):
        for j in range(i+1, n):
            ari, nmi = compute_stability_metrics(solution_list[i], solution_list[j], feature_order)
            aris.append(ari)
            nmis.append(nmi)
    return np.mean(aris), np.std(aris), np.mean(nmis), np.std(nmis)

def compute_pairwise_consistency(solution_list):
    """
    Berechnet pairwise Cluster-Konsistenz (durchschnittlicher Match-Prozentsatz) über eine Liste von Clusterlösungen.
    """
    consistencies = []
    n = len(solution_list)
    for i in range(n):
        for j in range(i+1, n):
            cons = average_cluster_consistency(solution_list[i], solution_list[j])
            consistencies.append(cons)
    return np.mean(consistencies), np.std(consistencies)

def average_cluster_consistency(clusters_1, clusters_2):
    """
    Berechnet für einen Vergleich zweier Clusterlösungen den durchschnittlichen Match-Prozentsatz.
    Für jeden Cluster in clusters_1 wird der beste Matching-Cluster in clusters_2 gesucht.
    """
    match_percentages = []
    for cluster_id, features_1 in clusters_1.items():
        best_match = max(clusters_2.items(), key=lambda x: len(set(features_1).intersection(set(x[1]))))
        match_percentage = len(set(features_1).intersection(set(best_match[1]))) / len(features_1) * 100
        match_percentages.append(match_percentage)
    return np.mean(match_percentages) if match_percentages else np.nan

def get_detailed_cluster_consistency(clusters_1, clusters_2):
    """
    Für jeden Cluster in clusters_1 wird der beste Matching-Cluster in clusters_2 gesucht.
    Gibt ein Dictionary zurück: {orig_cluster: (matched_cluster, match_percentage)}
    """
    details = {}
    for cluster_id, features_1 in clusters_1.items():
        best_match = max(clusters_2.items(), key=lambda x: len(set(features_1).intersection(set(x[1]))))
        match_percentage = len(set(features_1).intersection(set(best_match[1]))) / len(features_1) * 100
        details[cluster_id] = (best_match[0], match_percentage)
    return details

# -----------------------------
# 4. Funktion: Gruppenspezifisches Rauschen hinzufügen
# -----------------------------
def add_noise_to_group(df_group, noise_factor=0.1):
    return add_noise_to_data(df_group, noise_factor=noise_factor)

# -----------------------------
# Simulation: Für eine gegebene Untergruppe
# -----------------------------
def simulate_state_for_group(df_original, key, noise_factor=0.1, n_reps=100, state="global"):
    """
    Simuliert n_reps Clusterlösungen für eine gegebene Subgruppe (key) im Zustand:
      - "global": Es wird global Rauschen zum gesamten Datensatz hinzugefügt.
      - "group": Es wird nur in der Subgruppe Rauschen hinzugefügt.
    Gibt eine Liste von Clusterlösungen zurück.
    """
    solutions = []
    group_orig = df_original[(df_original["Verletzungsstatus"] == key[0]) & (df_original["Geschlecht_weiblich"] == key[1])]
    for _ in range(n_reps):
        if state == "global":
            df_global_noisy = add_noise_to_data(df_original, noise_factor)
            group = df_global_noisy[(df_global_noisy["Verletzungsstatus"] == key[0]) & (df_global_noisy["Geschlecht_weiblich"] == key[1])]
        elif state == "group":
            group = add_noise_to_group(group_orig, noise_factor)
        else:
            raise ValueError("state muss 'global' oder 'group' sein.")
        clusters = perform_clustering(group, max_clusters=4)
        solutions.append(clusters)
    return solutions

# -----------------------------
# 5. Main-Funktion
# -----------------------------
def main():
    # 5.1 Daten laden
    file_path = r"K:\Team\Böhmer_Michael\TSA\ML\Basistabelle\Basistabelle_ML_neu.xlsx"
    df_original = pd.read_excel(file_path)
    print("Original Data Shape:", df_original.shape, "\n")
    
    # Baseline: Original Cluster (deterministisch)
    clusters_original = get_group_clusters(df_original)
    
    # --- ERSTER DURCHLAUF: Ausgabe von Variablenlisten und Visualisierungen ---
    print("==== ERSTER DURCHLAUF: Variablenlisten und Visualisierungen ====\n")
    
    # Global verrauschte Cluster (ein Durchlauf)
    df_global_noisy = add_noise_to_data(df_original, noise_factor=0.1)
    clusters_global_noisy = get_group_clusters(df_global_noisy)
    
    # Gruppenspezifisch verrauschte Cluster (ein Durchlauf)
    clusters_group_specific = {}
    for key, group in df_original.groupby(["Verletzungsstatus", "Geschlecht_weiblich"]):
        group = group.reset_index(drop=True)
        group_noisy = add_noise_to_group(group, noise_factor=0.1)
        clusters_group_specific[key] = perform_clustering(group_noisy, max_clusters=4)
    
    # Ausgabe der Clusterzuordnungen (Variablenlisten)
    print("--- Cluster Zuordnungen: Original ---\n")
    for key, clusters in clusters_original.items():
        print(map_group_label(key))
        for cl, cols in clusters.items():
            print("  Cluster", cl, ":", cols)
        print("")
    
    print("--- Cluster Zuordnungen: Global Noisy ---\n")
    for key, clusters in clusters_global_noisy.items():
        print(map_group_label(key))
        for cl, cols in clusters.items():
            print("  Cluster", cl, ":", cols)
        print("")
    
    print("--- Cluster Zuordnungen: Gruppenspezifisch Noisy ---\n")
    for key, clusters in clusters_group_specific.items():
        print(map_group_label(key))
        for cl, cols in clusters.items():
            print("  Cluster", cl, ":", cols)
        print("")
    
    # Visualisierung für eine Beispieluntergruppe
    example_key = list(clusters_original.keys())[0]
    group_orig = df_original[(df_original["Verletzungsstatus"]==example_key[0]) & 
                             (df_original["Geschlecht_weiblich"]==example_key[1])]
    group_global_noisy = df_global_noisy[(df_global_noisy["Verletzungsstatus"]==example_key[0]) & 
                                         (df_global_noisy["Geschlecht_weiblich"]==example_key[1])]
    group_group_noisy = add_noise_to_group(group_orig, noise_factor=0.1)
    clusters_example = perform_clustering(group_group_noisy, max_clusters=4)
    
    visualize_clusters(group_orig, clusters_original[example_key],
                         title=f"Original Cluster ({map_group_label(example_key)})")
    visualize_clusters(group_global_noisy, clusters_global_noisy[example_key],
                         title=f"Global Noisy Cluster ({map_group_label(example_key)})")
    visualize_clusters(group_group_noisy, clusters_example,
                         title=f"Gruppenspezifisch Noisy Cluster ({map_group_label(example_key)})")
    
    # --- SIMULATION (50 Wiederholungen) zur Aggregation der Metriken ---
    n_reps = 50
    sim_results = {}   # Speichert für jede Subgruppe: { 'original': (ARI, NMI, Consistency),
                      #  'global': (mean_ARI, std_ARI, mean_NMI, std_NMI, mean_cons, std_cons),
                      #  'group': (mean_ARI, std_ARI, mean_NMI, std_NMI, mean_cons, std_cons) }
    sim_details = {}   # Für die detaillierte Konsistenz: {key: {comp: {orig_cluster: {'matched': [], 'consistency': []}} } }
    
    for key in clusters_original.keys():
        sim_results[key] = {
            'original': (1.0, 0.0, 1.0, 0.0, 100.0, 0.0),
            'global': None,
            'group': None
        }
        sim_details[key] = {
            'orig_vs_global': {},
            'orig_vs_group': {},
            'global_vs_group': {}
        }
        
        feature_order = get_feature_order(df_original[(df_original["Verletzungsstatus"]==key[0]) & 
                                                      (df_original["Geschlecht_weiblich"]==key[1])])
        
        # Global verrauscht
        global_solutions = simulate_state_for_group(df_original, key, 0.1, n_reps, state="global")
        ari_g, std_ari_g, nmi_g, std_nmi_g = compute_pairwise_metrics(global_solutions, feature_order)
        cons_g, std_cons_g = compute_pairwise_consistency(global_solutions)
        
        # Gruppenspezifisch verrauscht
        group_solutions = simulate_state_for_group(df_original, key, 0.1, n_reps, state="group")
        ari_grp, std_ari_grp, nmi_grp, std_nmi_grp = compute_pairwise_metrics(group_solutions, feature_order)
        cons_grp, std_cons_grp = compute_pairwise_consistency(group_solutions)
        
        sim_results[key]['global'] = (ari_g, std_ari_g, nmi_g, std_nmi_g, cons_g, std_cons_g)
        sim_results[key]['group'] = (ari_grp, std_ari_grp, nmi_grp, std_nmi_grp, cons_grp, std_cons_grp)
        
        # Aggregiere detaillierte Konsistenzdaten
        for comp, sols in zip(['orig_vs_global', 'orig_vs_group'], [global_solutions, group_solutions]):
            details = [get_detailed_cluster_consistency(clusters_original[key], sol) for sol in sols]
            # Nun berechnen wir für jeden Originalcluster die Liste der matched Werte und Consistencies
            for d in details:
                for orig_cluster, (matched, cons) in d.items():
                    if orig_cluster not in sim_details[key][comp]:
                        sim_details[key][comp][orig_cluster] = {'matched': [], 'consistency': []}
                    sim_details[key][comp][orig_cluster]['matched'].append(matched)
                    sim_details[key][comp][orig_cluster]['consistency'].append(cons)
        
        # Für global_vs_group Vergleich
        details = [get_detailed_cluster_consistency(sol1, sol2) for sol1, sol2 in zip(global_solutions, group_solutions)]
        for d in details:
            for orig_cluster, (matched, cons) in d.items():
                if orig_cluster not in sim_details[key]['global_vs_group']:
                    sim_details[key]['global_vs_group'][orig_cluster] = {'matched': [], 'consistency': []}
                sim_details[key]['global_vs_group'][orig_cluster]['matched'].append(matched)
                sim_details[key]['global_vs_group'][orig_cluster]['consistency'].append(cons)
    
    # Aggregierte Ausgabe der stabilitätsmetriken
    print("=== Aggregierte Stabilitätsmetriken über", n_reps, "Wiederholungen ===\n")
    for key, metrics in sim_results.items():
        print(map_group_label(key))
        orig = metrics['original']
        print(f"  original:             ARI = {orig[0]:.3f} ± {orig[1]:.3f}, NMI = {orig[2]:.3f} ± {orig[3]:.3f}, Consistency = {orig[4]:.2f}% ± {orig[5]:.2f}%")
        glob = metrics['global']
        print(f"  global verrauscht:    ARI = {glob[0]:.3f} ± {glob[1]:.3f}, NMI = {glob[2]:.3f} ± {glob[3]:.3f}, Consistency = {glob[4]:.2f}% ± {glob[5]:.2f}%")
        grp = metrics['group']
        print(f"  gruppenspezifisch:    ARI = {grp[0]:.3f} ± {grp[1]:.3f}, NMI = {grp[2]:.3f} ± {grp[3]:.3f}, Consistency = {grp[4]:.2f}% ± {grp[5]:.2f}%")
        print("")
    
    # Aggregierte Ausgabe der detaillierten Cluster-Konsistenz
    print("=== Aggregierte Detaillierte Cluster-Konsistenz (Übereinstimmungsprozente) ===\n")
    for key, comp_dict in sim_details.items():
        print(map_group_label(key))
        for comp, clusters_dict in comp_dict.items():
            print(f"  {comp}:")
            for orig_cluster, values in clusters_dict.items():
                cons_mean = np.mean(values['consistency'])
                cons_std = np.std(values['consistency'])
                mode = Counter(values['matched']).most_common(1)[0][0]
                print(f"    Orig. Cluster {orig_cluster}: {mode} (Übereinstimmung: {cons_mean:.2f}% ± {cons_std:.2f}%)")
            print("")
        print("")
    
if __name__ == '__main__':
    main()


In [37]:
!git add Test_ML.ipynb

In [31]:
!git remote set-url origin https://github.com/michi1308/Test_ML.git


In [39]:
!git commit -m "Datenaugmentation Validierung mit Rauschen"

[master 8a9f928] Datenaugmentation Validierung mit Rauschen
 1 file changed, 11 insertions(+), 3 deletions(-)


In [41]:
!git status

On branch master
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   Motum_ML.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.ipynb_checkpoints/
	Datenaugmentation.ipynb
	Datenexport_Skript_2.0.ipynb
	Exzentrik_finale_Version.ipynb
	Hyperparameter.ipynb
	Isokinetik_finale_Version.ipynb
	Isometrie_finale_Version.ipynb
	ML_Mutual_Information_Korrelationsmatrix.ipynb
	Mergetabelle_NEU.ipynb
	PCA.ipynb
	PCA_TSA.ipynb
	Sprung_Excel_Merge.ipynb
	TSA_grafik.ipynb
	Test.ipynb
	Untitled2.ipynb
	Untitled3.ipynb
	Vergleich_TSA_ML.ipynb
	algorithmus_signifikante_werte.ipynb
	best_model.py
	git_hub.ipynb
	gradient_boosting_optimieren.ipynb
	maestroni_faktoren.ipynb
	maestroni_statistik_deskriptiv.ipynb
	ml.ipynb
	ml_algorithmus_finden.ipynb
	motum_ml_feature_selection/
	stacking_tsa.ipynb

no changes added to commit (use "git add" an

In [43]:
!git remote -v

origin	https://github.com/michi1308/Test_ML.git (fetch)
origin	https://github.com/michi1308/Test_ML.git (push)


In [45]:
!git push origin master

To https://github.com/michi1308/Test_ML.git
   511cd20..8a9f928  master -> master
